In [ ]:
!pip install keras-tuner


In [14]:
import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras import layers
import numpy as np


In [15]:
print(tf.__version__)

2.9.1


In [16]:
# load data
import pickle
x = pickle.load(open("features100-X.pkl", "rb"))
y = pickle.load(open("labels100-Y.pkl", "rb"))

In [ ]:
# k fold cross validation
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(x)

# split data
for train_index, test_index in kf.split(x):
    train_images, test_images = x[train_index], x[test_index]
    train_labels, test_labels = y[train_index], y[test_index]


In [ ]:
# (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
# scale down images
train_images = train_images / 255.0
test_images = test_images / 255.0


In [ ]:
train_images[0].shape

(100, 100, 3)

In [ ]:
# train_images=train_images.reshape(len(train_images),28,28,1)
# test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def buildmodel(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
            activation='relu',
            input_shape=(100,100,3)
        ),
        keras.layers.Conv2D(
            filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
            kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
            activation='relu'
        ),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
            activation='relu'
        ),
        keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters



C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_97292\4188932029.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [12]:
tuner_search=RandomSearch(buildmodel,
                            objective='val_accuracy',
                            max_trials=5,directory='output',project_name="FERsystem")

INFO:tensorflow:Reloading Oracle from existing project output\FERsystem\oracle.json


In [13]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
80                |?                 |conv_1_filter
3                 |?                 |conv_1_kernel
48                |?                 |conv_2_filter
5                 |?                 |conv_2_kernel
64                |?                 |dense_1_units
0.001             |?                 |learning_rate

Epoch 1/3
879/879 [==============================] - 1792s 2s/step - loss: 1.7295 - accuracy: 0.3404 - val_loss: 15.1113 - val_accuracy: 0.0086
Epoch 2/3
  8/879 [..............................] - ETA: 29:35 - loss: 1.5311 - accuracy: 0.4062

KeyboardInterrupt: 

In [ ]:
model = tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)